In [1]:
import yaml
import numpy as np
import torch
import lightning as L
from lightning.pytorch.callbacks import RichModelSummary, DeviceStatsMonitor, EarlyStopping, ModelCheckpoint
import wandb
from lightning.pytorch.loggers import WandbLogger
import matplotlib.pyplot as plt
# torch.cuda.empty_cache()
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1" # only when profiling

pathToProject = 'skola/Fourier-Neural-Operator'
gpu = False # Set to true if CUDA is available

%cd 
%cd $pathToProject/src/fno/model
from Utilities import *
from FNOModel import *

C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator\src\fno\model


In [2]:
# Import configurations

%cd
%cd $pathToProject/src/fno/configs
# path = 'NavierStokes_V1e-3_N5000_T50_32x32-N5000.yml'
path = 'NavierStokes_V1e-3_N5000_T50_32x32-N40.yml'

with open(path) as file:
    vars = yaml.load(file, Loader=yaml.FullLoader)
    globals().update(vars)

C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator\src\fno\configs


In [3]:
# Instantiate FNO and data loaders

%cd
%cd $pathToProject
train_loader, val_loader, test_loader = npyToTorchDataLoader(DATA_PATH, batchSize=batch_size, normalize=True, timeDistribution=timeDistribution, distribution=distribution)
time_steps_x = train_loader.dataset[0][0].shape[-1]
input_size = time_steps_x + len(('x', 'y', 't'))
restart_at_epoch_n = epochs // (2 * n_restarts + 1) # Moved up 1 cell so we can restart Kernel and load an old model without training
fno_model = FNOModel(in_neurons, hidden_neurons, out_neurons, modesSpace, modesTime, time_padding, input_size, learning_rate, restart_at_epoch_n, train_loader, loss_function)

C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator


c:\Users\erikn\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Train FNO

# You have to login to wandb to run the script (wandb logs the training). Follow these steps to proceed:
# 1. Create an account on wandb.ai and copy your personal API key
# 2. Prompt the terminal: wandb login
# 3. Paste your API key and hit enter

'''
wandb_logger = WandbLogger(project='Fourier Neural Operator')
wandb_logger.experiment.config["batch_size"] = batch_size

print('Number of parameters: ', count_params(fno_model))
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor='val_' + loss_function + '_loss', mode="min")
early_stopping = EarlyStopping(monitor='val_' + loss_function + '_loss', patience=12)
CALLBACKS=[checkpoint_callback, early_stopping]
trainer = L.Trainer(accelerator="auto", devices="auto", strategy="auto", max_epochs=epochs, callbacks=CALLBACKS, logger=wandb_logger) #DeviceStatsMonitor(cpu_stats=True)
trainer.fit(fno_model, train_loader, val_loader)

wandb.finish()
'''

In [11]:
# Save trained FNO

'''
%cd
%cd $pathToProject/src/fno/model
model = 'trained-model-32x32-500epoch-v1-3-n4'
trainer.save_checkpoint('model-checkpoint/'+model+'.ckpt')
torch.save(fno_model.state_dict(), "model-state-dicts/"+model+".pt")
'''

C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator\src\fno\model


In [ ]:
# Test trained FNO

# trainer.test(fno_model, dataloaders=test_loader)

In [4]:
# Instantiate saved FNO

%cd
%cd $pathToProject/src/fno/model/model-state-dicts
loaded_model = FNOModel(in_neurons, hidden_neurons, out_neurons, modesSpace, modesTime, time_padding, input_size, learning_rate, restart_at_epoch_n, train_loader, loss_function)
loaded_model.load_state_dict(torch.load("trained-model-32x32-500epoch-v1-3-n3.pt")) # This is our best trained model
del train_loader
del val_loader
del test_loader

C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator\src\fno\model\model-state-dicts


In [6]:
# Evaluate FNO on different spatial resolutions. 
# A 95% confidence interval is computed here to determine the certainties of the predicitions for both the FNO and a bicubic interpolation.

%cd
%cd $pathToProject/src/data/datasets
DATA_PATH = 't1-t50VorticityZ-32x32-v1e-3-T50-N40.npy'
# DATA_PATH = 't1-t50VorticityZ-128x128-v1e-3-T50-N20.npy'
# DATA_PATH = 't1-t50VorticityZ-1024x1024-v1e-3-T50-N10.npy'

batch_size = 1
# torch.cuda.empty_cache()
y_hat_upsampled, y_hat_fno, y = CalculateStatistics(DATA_PATH, loaded_model, batchSize=batch_size, gif=False, precision='16-mixed', gpu=gpu)

c:\Users\erikn\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\accelerator_connector.py:508: UserWarning: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


C:\Users\erikn
C:\Users\erikn\skola\Fourier-Neural-Operator\src\data\datasets


Predicting DataLoader 0: 100%|██████████| 40/40 [06:00<00:00,  9.01s/it]
FNO: average loss: 0.00851534865796566,  95% confidence interval: 0.0023974073119461536
Predicting DataLoader 0: 100%|██████████| 40/40 [05:42<00:00,  8.57s/it]
Bicubic interpolation: average loss: 0.00851534865796566,  95% confidence interval: 0.0023974073119461536


In [ ]:
# Saving 2 tensors (true, prediction) of one flow of a specified resolution to be able to create gifs

%cd
%cd $pathToProject/src/data/datasets

spatial_res = 32
temp_res = 1
DATA_PATH = 't1-t50VorticityZ-{}x{}-v1e-3-T50-N40.npy'.format(spatial_res, spatial_res)
batch_size = 1
gif = True
y_hat_upsampled, y_hat_fno, y = CalculateStatistics(DATA_PATH, loaded_model, batchSize=batch_size, precision='32', gif=gif, temporal_pred=False, temp_res=temp_res, gpu=gpu)

%cd
%cd $pathToProject/src/fno/output-flows
if gif:
    # torch.save(y_hat_upsampled, 'biqube_{}x{}_result.pt'.format(res,res))
    torch.save(y_hat_fno, '{}x{}/fno_{}x{}_result_t{}_TEST.pt'.format(spatial_res, spatial_res, spatial_res, spatial_res, temp_res))    # Predicted flow by FNO
    torch.save(y, '{}x{}/true_{}x{}_result_t{}_TEST.pt'.format(spatial_res, spatial_res, spatial_res, spatial_res, temp_res))           # True flow